In [1]:
import sys
import asyncio

if "pyodide" in sys.modules:
    import micropip
    
    async def install_widgets():
        await micropip.install("ipywidgets")
    
    asyncio.run(install_widgets())

# Now we can safely import ipywidgets
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

# -----------------------------------------------------------------------------
# Pricing data: Assume these are DAILY rates in USD for each machine type.
# -----------------------------------------------------------------------------
NODE_PRICING = {
    'n2-highcpu-32': 27.93,  # daily price for n2-highcpu-32
    'n2-highcpu-96': 83.73,  # daily price for n2-highcpu-96
    'n2-highmem-32': 50.31,  # daily price for n2-highmem-32
}

# -----------------------------------------------------------------------------
# Function: calculate_total_cost
# Description: Calculates cost for the specified machine type, CPUs per user,
#              number of students, and number of hours. Converts daily cost to
#              hourly cost by dividing by 24.
# -----------------------------------------------------------------------------
def calculate_total_cost(node_type, num_cpus, num_students, num_hours):
    """
    Calculate the cost of running workloads for num_hours, given:
      - node_type: the machine type (e.g., 'n2-highcpu-32')
      - num_cpus: CPUs needed per student
      - num_students: total number of students (or users)
      - num_hours: total hours of usage
    """

    # Extract the CPU count from the node type, e.g. "n2-highcpu-32" -> 32
    node_capacity = int(node_type.split('-')[-1])
    
    # How many students can run on one node?
    students_per_node = node_capacity / num_cpus
    
    # How many nodes total?
    total_nodes = num_students / students_per_node
    
    # Retrieve daily cost from the pricing dictionary
    daily_cost = NODE_PRICING[node_type]
    
    # Convert daily cost to an hourly rate
    hourly_cost = daily_cost / 24.0
    
    # Total cost = number_of_nodes * hourly_rate * hours
    total_cost = total_nodes * hourly_cost * num_hours
    
    return total_nodes, total_cost

# -----------------------------------------------------------------------------
# Event callback: on_button_click
# -----------------------------------------------------------------------------
def on_button_click(b):
    node_type = node_type_dropdown.value
    num_cpus = int(num_cpus_dropdown.value)
    num_students = int(students_input.value)
    num_hours = int(num_hours_input.value)
    
    total_nodes, total_cost = calculate_total_cost(node_type, num_cpus, num_students, num_hours)
    
    # Update the result widget
    result_text.value = (
        f"Total nodes allocated: {total_nodes:.2f}\n"
        f"Total cost for {num_hours} hours: ${total_cost:.2f}"
    )

# -----------------------------------------------------------------------------
# Create the Widgets
# -----------------------------------------------------------------------------
node_type_dropdown = widgets.Dropdown(
    options=['n2-highcpu-32', 'n2-highcpu-96', 'n2-highmem-32'],
    value='n2-highcpu-32',
    description='Node Type:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

num_cpus_dropdown = widgets.Dropdown(
    options=['2', '4'],
    value='4',
    description='CPUs per Student:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

students_input = widgets.IntText(
    value=1200,
    description='Total Students:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

num_hours_input = widgets.IntText(
    value=24,
    description='Number of Hours:',
    style={'description_width': 'auto'},
    layout=Layout(width='50%', padding='1em')
)

calculate_button = widgets.Button(
    description='Calculate Total Cost',
    button_style='success',
    layout=Layout(width='50%')
)

result_text = widgets.Textarea(
    value='',
    description='Result:',
    disabled=True,
    layout=Layout(width='50%', height='4em', padding='1em'),
    style={'description_width': 'initial', 'text-align': 'center'}
)

# Attach the event handler to the button
calculate_button.on_click(on_button_click)

# -----------------------------------------------------------------------------
# Layout the Widgets
# -----------------------------------------------------------------------------
input_columns = widgets.VBox(
    [node_type_dropdown, num_cpus_dropdown, students_input, num_hours_input, calculate_button],
    layout=Layout(width='100%', padding='1em')
)

final_layout = widgets.VBox(
    [input_columns, result_text],
    layout=Layout(padding='1em')
)

# -----------------------------------------------------------------------------
# Display Widgets (Requires a Jupyter environment)
# -----------------------------------------------------------------------------
display(final_layout)
